In [1]:
import csv
import pandas as pd
import sys

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
__RESULTS_FILE_DTYPES = {"DYAD": "category", "WORD": "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

infile_path = r"D:\Users\tcshore\Downloads\wordscores.tsv"
cv_results = read_results_file(infile_path, "windows-1252")

Reading "D:\Users\tcshore\Downloads\wordscores.tsv" using encoding "windows-1252".


In [2]:
dyad_ids = tuple(sorted(frozenset(cv_results["DYAD"].unique())))
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], len(dyad_ids)),
      file=sys.stderr)
entity_ids = frozenset(cv_results["ENTITY"].unique())
print("Found {} unique entity IDs.".format(len(entity_ids)), file=sys.stderr)

Read 1206840 cross-validation results for 39 dyad(s).
Found 20 unique entity IDs.


In [3]:
def are_all_entities_represented(df: pd.DataFrame, entity_ids) -> bool:
	"""
	Checks if all entities are represented for each individual token in each utterance in the dataframe.
    
	:param df: The dataframe to check.
	:param entity_ids: A collection of all unique entity IDs.
	:return: true iff for each token in each utterance there is one row for each entity ID.
	"""
	utt_toks = df.groupby(
		("CROSS_VALIDATION_ITER", "DYAD", "ROUND", "UTT_START_TIME", "UTT_END_TIME", "WORD", "TOKEN_SEQ_ORDINALITY"), as_index=False)
	print("Found {} utterance tokens for all cross-validations.".format(len(utt_toks)), file=sys.stderr)
	# Check if there is a row for each entity (possible referent) for each token
	return all(utt_toks.apply(lambda group: is_collection_equivalent(group["ENTITY"], entity_ids)))

def is_collection_equivalent(c1, c2) -> bool:
	c1_len = len(c1)
	c2_len = len(c2)
	return c1_len == c2_len and all(elem in c2 for elem in c1)

assert are_all_entities_represented(cv_results, entity_ids)

Found 60342 utterance tokens for all cross-validations.


In [4]:
def find_target_ref_rows(df: pd.DataFrame) -> pd.DataFrame:
    result = df.loc[df["IS_TARGET"] == True]
    result_row_count = result.shape[0]
    complement_row_count = df.loc[~df.index.isin(result.index)].shape[0]
    assert result_row_count + complement_row_count == df.shape[0]
    print("Found {} nontarget rows and {} target rows. Ratio: {}".format(result_row_count, complement_row_count, complement_row_count / float(result_row_count)), file=sys.stderr)
    return result

cv_results = find_target_ref_rows(cv_results)

Found 60342 nontarget rows and 1146498 target rows. Ratio: 19.0


In [57]:
import math
from typing import List, Tuple

import keras.preprocessing.sequence
import numpy as np
from sklearn.preprocessing import OneHotEncoder

class TokenSequenceFactory(object):

	def __init__(self, max_len: int):
		self.__max_len = max_len
		self.__max_len_divisor = float(max_len)

	def __call__(self, df: pd.DataFrame) -> Tuple[List[np.array], List[np.array]]:
		"""
		Creates a sequence of sequences of tokens, each representing an utterance, each of which thus causes an "interruption" in the chain
		so that e.g. the first token of one utterance is not learned as dependent on the last token of the utterance preceding it.
		:param df: The DataFrame to process.
		:return: Paired lists of 2D numpy arrays, each representing a sequence of datapoints which represents an utterance and the corresponding scores to predict.
		"""
        # https://stackoverflow.com/a/47815400/1391325
		df.sort_values("TOKEN_SEQ_ORDINALITY", inplace=True)
		sequences = df.groupby(("CROSS_VALIDATION_ITER", "DYAD", "ROUND", "UTT_START_TIME", "UTT_END_TIME", "ENTITY"),
							   as_index=False)
		#sequences.apply(lambda seq : self.create_word_forms(seq["ONEHOT_WORD_LABEL"].values))
        
		word_seqs = []
		score_seqs = []
		max_len_divisor = float(self.__max_len)
		split_seq_scores = sequences.apply(self.__split_row_values)
		for word_seq, score_seq in split_seq_scores:
			word_seqs.extend(word_seq)
			score_seqs.extend(score_seq)
		assert max(len(seq) for seq in word_seqs) <= self.__max_len
		return word_seqs, score_seqs

	def __split_row_values(self, df: pd.DataFrame) -> Tuple[np.array, np.array]:
		seq_words = df["WORD"].values
		seq_scores = df["PROBABILITY"].values
        
		partition_count = math.ceil(len(seq_words) / self.__max_len_divisor)
		split_seq_words = np.array_split(seq_words, partition_count)
		split_seq_scores = np.array_split(seq_scores, partition_count)
		return split_seq_words, split_seq_scores

desired_seq_len = 4
print("Splitting token sequences.", file=sys.stderr)
token_seq_factory = TokenSequenceFactory(desired_seq_len)
word_seqs, score_seqs = token_seq_factory(cv_results)
print("Split data into {} token sequences with a maximum sequence length of {}.".format(len(word_seqs),
                                                                                        desired_seq_len),
      file=sys.stderr)

Splitting token sequences.
Split data into 20740 token sequences with a maximum sequence length of 4.


In [64]:
from typing import Sequence

def pad_sequence(word_score_seq: Tuple[np.array, np.array], min_length: int) -> Tuple[np.array, np.array]:
	word_seq, score_seq = word_score_seq
	word_count = len(word_seq)
	assert word_count == len(score_seq)
	length_diff = min_length - word_count
	if length_diff > 0:
		# NOTE: creating an intermediate tuple is necessary
		padding_words = np.full(length_diff, "__PADDING__")
		padded_word_seq = np.concatenate((padding_words, word_seq), axis=0)
		assert len(padded_word_seq) == min_length
		padding_scores = np.full(length_diff, 0.0)
		padded_score_seq = np.concatenate((padding_scores, score_seq), axis=0)
		assert len(padded_score_seq) == min_length
		result = padded_word_seq, padded_score_seq
	else:
		result = word_seq, score_seq

	return result

def pad_sequences(word_score_seqs : Sequence[Tuple[np.array, np.array]], min_length: int):
    for word_score_seq in word_score_seqs:
        print(pad_sequence(word_score_seq, min_length))

            
pad_sequences(zip(word_seqs, score_seqs), desired_seq_len)

(array(['ah', 'okay', 'now', 'I'], dtype=object), array([ 0.60987518,  0.51752287,  0.44109063,  0.47596564]))
(array(['see', 'a', 'selected', 'piece'], dtype=object), array([ 0.49421434,  0.45865621,  0.12580566,  0.46943467]))
(array(['__PADDING__', '__PADDING__', 'all', 'right'], dtype=object), array([ 0.       ,  0.       ,  0.3956217,  0.3559954]))
(array(['waiting', 'for', 'other', 'player'], dtype=object), array([  4.48115780e-16,   4.28518916e-01,   5.58960072e-01,
         1.83746008e-22]))
(array(['to', 'select', 'a', 'piece'], dtype=object), array([ 0.52388737,  0.39941035,  0.45865621,  0.46943467]))
(array(['to', 'move', 'all', 'right'], dtype=object), array([  5.23887371e-01,   2.43874664e-13,   3.95621699e-01,
         3.55995401e-01]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'okay'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.51752287]))
(array(['so', 'so', 'it', "'s"], dtype=object), array([ 0.36887535,  0.36887535,  0.48151029,  0.482

(array(['__PADDING__', 'the', 'yellow', 'one'], dtype=object), array([ 0.        ,  0.50487925,  0.8713726 ,  0.37393538]))
(array(['__PADDING__', '__PADDING__', 'yeah', 'exactly'], dtype=object), array([ 0.        ,  0.        ,  0.4564016 ,  0.30998474]))
(array(['__PADDING__', 'it', "'s", 'the'], dtype=object), array([ 0.        ,  0.46676806,  0.44782556,  0.50487925]))
(array(['__PADDING__', '__PADDING__', 'yellow', 'thing'], dtype=object), array([ 0.       ,  0.       ,  0.8713726,  0.6281531]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'yeah'], dtype=object), array([ 0.       ,  0.       ,  0.       ,  0.4564016]))
(array(['__PADDING__', '__PADDING__', 'all', 'right'], dtype=object), array([ 0.        ,  0.        ,  0.49285751,  0.42723517]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'okay'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.49051975]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'okay'], dtype=object), array([ 0.   

(array(['the', 'red', 'piece', 'at'], dtype=object), array([ 0.45500616,  0.96455551,  0.48309168,  0.29075322]))
(array(['__PADDING__', 'the', 'top', 'right'], dtype=object), array([ 0.        ,  0.45500616,  0.89139286,  0.71232058]))
(array(['__PADDING__', '__PADDING__', 'top', 'right'], dtype=object), array([ 0.        ,  0.        ,  0.89139286,  0.71232058]))
(array(['__PADDING__', '__PADDING__', 'top', 'right'], dtype=object), array([ 0.        ,  0.        ,  0.89139286,  0.71232058]))
(array(['__PADDING__', 'so', 'it', 'looks'], dtype=object), array([ 0.        ,  0.4698081 ,  0.49801141,  0.44950114]))
(array(['__PADDING__', 'like', 'a', 'kind'], dtype=object), array([ 0.        ,  0.48415977,  0.47555488,  0.49392148]))
(array(['__PADDING__', 'of', 'like', 'a'], dtype=object), array([ 0.        ,  0.58562647,  0.48415977,  0.47555488]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'you'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.45009602]))
(ar

(array(["'s", 'a', 'green', 'V'], dtype=object), array([ 0.47899825,  0.4390844 ,  0.74315815,  0.95419301]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'yeah'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.41117961]))
(array(['so', 'the', 'place', 'where'], dtype=object), array([ 0.63588433,  0.55208019,  0.00179285,  0.38297503]))
(array(['I', 'selected', 'the', 'V'], dtype=object), array([ 0.74151189,  0.98968396,  0.55208019,  0.06963865]))
(array(['right', 'on', 'the', 'opposite'], dtype=object), array([  4.91796480e-01,   6.84771542e-01,   5.52080191e-01,
         6.50808105e-17]))
(array(['end', 'the', 'left', 'side'], dtype=object), array([ 0.99999951,  0.55208019,  0.92063744,  0.86775964]))
(array(['of', 'it', 'a', 'weird'], dtype=object), array([ 0.61767955,  0.5715637 ,  0.59060528,  0.90453653]))
(array(['looking', 'blue', 'color', 'thingy'], dtype=object), array([  4.41160145e-20,   8.43337824e-01,   5.38798835e-01,
         2.41419815e-01]))


(array(['__PADDING__', '__PADDING__', '__PADDING__', 'yeah'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.47353393]))
(array(['__PADDING__', 'yeah', 'yeah', 'nice'], dtype=object), array([ 0.        ,  0.47353393,  0.47353393,  0.38538197]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'awesome'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.98871269]))
(array(['okay', 'it', "'s", 'the'], dtype=object), array([ 0.4982041 ,  0.48694979,  0.48873926,  0.49350809]))
(array(['__PADDING__', 'big', 'weird', 'one'], dtype=object), array([ 0.        ,  0.59426673,  0.63816704,  0.54247137]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'mm-hmm'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.65563051]))
(array(['__PADDING__', 'in', 'the', 'bottom'], dtype=object), array([ 0.        ,  0.52207626,  0.49350809,  0.8619188 ]))
(array(['__PADDING__', '__PADDING__', 'the', 'big'], dtype=object), array([ 0.        ,  0.        

...

(array(['yeah', 'it', "'s", 'the'], dtype=object), array([ 0.51438258,  0.49229513,  0.46209475,  0.52472751]))
(array(['same', 'blue', 'yellow', 'spaceship'], dtype=object), array([ 0.69518942,  0.02012393,  0.90229993,  0.89179557]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'yellow'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.90229993]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'yeah'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.51438258]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'oh'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.45979892]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'cool'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.51435167]))
(array(['__PADDING__', 'the', 'green', 'V'], dtype=object), array([ 0.        ,  0.47291757,  0.79788792,  0.90656318]))
(array(['__PADDING__', 'it', "'s", 'the'], dtype=object), array([ 0.        ,  0.47369939, 

(array(['or', 'bird', 'or', 'whatever'], dtype=object), array([ 0.38840449,  0.72865404,  0.38840449,  0.66999066]))
(array(['the', 'big', 'yellow', 'thing'], dtype=object), array([ 0.50452623,  0.56912103,  0.61561723,  0.74134284]))
(array(['with', 'lots', 'of', 'triangles'], dtype=object), array([  4.37684011e-01,   1.22790018e-08,   5.96528594e-01,
         7.56281493e-01]))
(array(['the', 'small', 'blue', 'TV'], dtype=object), array([ 0.50277968,  0.62561804,  0.74058884,  0.97643972]))
(array(['the', 'light', 'blue', 'emerald'], dtype=object), array([  4.88552249e-01,   3.76127934e-01,   7.42918031e-01,
         2.84771088e-10]))
(array(['__PADDING__', 'the', 'light', 'blue'], dtype=object), array([ 0.        ,  0.49850087,  0.75410024,  0.63284733]))
(array(['__PADDING__', 'mountain', 'very', 'small'], dtype=object), array([ 0.        ,  0.97788932,  0.2947244 ,  0.79484728]))
(array(['__PADDING__', 'the', 'small', 'kind'], dtype=object), array([ 0.        ,  0.48022423,  0.5177

(array(['okay', 'it', "'s", 'pink'], dtype=object), array([ 0.46824597,  0.51810128,  0.52866662,  0.89296717]))
(array(['and', 'it', "'s", 'a'], dtype=object), array([ 0.56199331,  0.51810128,  0.52866662,  0.61688262]))
(array(['little', 'bit', 'down', 'than'], dtype=object), array([  2.15342347e-01,   5.25152637e-01,   4.31057644e-01,
         7.27326217e-11]))
(array(['__PADDING__', 'the', 'bird', 'pink'], dtype=object), array([ 0.        ,  0.51830343,  0.16789492,  0.89296717]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'mm-hmm'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.44976004]))
(array(['do', 'you', 'see', 'a'], dtype=object), array([ 0.52174189,  0.54115676,  0.48757575,  0.61688262]))
(array(['__PADDING__', 'bird', 'pink', 'like'], dtype=object), array([ 0.        ,  0.16789492,  0.89296717,  0.5153262 ]))
(array(['__PADDING__', 'in', 'the', 'middle'], dtype=object), array([ 0.        ,  0.61361408,  0.51830343,  0.83370321]))
(array(['__PA

(array(['__PADDING__', '__PADDING__', 'light', 'green'], dtype=object), array([ 0.       ,  0.       ,  0.4963843,  0.9101263]))
(array(['the', 'crazy', 'grass', 'with'], dtype=object), array([  5.01865745e-01,   5.88106358e-18,   5.85001042e-01,
         5.40375119e-01]))
(array(['the', 'lighter', 'green', 'color'], dtype=object), array([ 0.50186575,  0.61212998,  0.57387434,  0.5894747 ]))
(array(['__PADDING__', 'the', 'coniferous', 'tree'], dtype=object), array([ 0.        ,  0.51305371,  0.56721412,  0.81865781]))
(array(['__PADDING__', '__PADDING__', 'in', 'blue'], dtype=object), array([ 0.        ,  0.        ,  0.55515466,  0.69684435]))
(array(['slanted', 'rectangle', 'with', 'two'], dtype=object), array([ 0.56765902,  0.71809349,  0.31655651,  0.56196587]))
(array(['__PADDING__', 'triangles', 'in', 'purple'], dtype=object), array([ 0.        ,  0.28575981,  0.54390796,  0.55192194]))
(array(['__PADDING__', '__PADDING__', 'the', 'diya'], dtype=object), array([ 0.        ,  0.  

(array(['I', 'think', 'it', "'s"], dtype=object), array([ 0.50510085,  0.34810138,  0.47873219,  0.50157333]))
(array(['__PADDING__', 'the', 'only', 'one'], dtype=object), array([ 0.        ,  0.47113876,  0.38356262,  0.53603122]))
(array(['__PADDING__', 'in', 'that', 'color'], dtype=object), array([ 0.        ,  0.48869541,  0.47375559,  0.48636616]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'yeah'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.47700633]))
(array(['__PADDING__', 'it', "'s", 'a'], dtype=object), array([ 0.        ,  0.50186542,  0.49432855,  0.4649161 ]))
(array(['__PADDING__', 'chicken', 'yellow', 'chicken'], dtype=object), array([ 0.        ,  0.98913663,  0.77436549,  0.98913663]))
(array(['__PADDING__', '__PADDING__', 'hen', 'or'], dtype=object), array([ 0.        ,  0.        ,  1.        ,  0.66274288]))
(array(['__PADDING__', '__PADDING__', '__PADDING__', 'bwok'], dtype=object), array([ 0.        ,  0.        ,  0.        ,  0.477

In [58]:
for seq in score_seqs:
    print("scoreseq")
    for s in seq:
        print("s")
        for something in s:
            print(something)


for seq in word_seqs:
    print("seq")
    for t in seq:
        print("t")
        for something in t:
            print(something)

scoreseq
s


TypeError: 'numpy.float64' object is not iterable

In [20]:
import itertools

from sklearn.preprocessing import LabelEncoder

all_words = tuple(itertools.chain(("__PADDING__", ), cv_results["WORD"].values))
print("Converting {} vocabulary entries to integer labels.".format(len(all_words)), file=sys.stderr)
# integer encode <https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/>
label_encoder = LabelEncoder()
label_encoder.fit(all_words)
for word_seq in word_seqs:
    integer_label_seq = label_encoder.transform(word_seq)
    print(integer_label_seq)
#integer_label_seqs = tuple(label_encoder.transform(word_seq) for word_seq in word_seqs)
#print(integer_label_seqs)

Converting 60343 vocabulary entries to integer labels.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[101 857 844  37]
[1059   85 1065  924]
[ 105 1023]
[1330  526  876  937]
[1246 1064   85  924]
[1246  805  105 1023]
[857]
[1118 1118  672    5]
[ 105 1023]
[672   5  85 928]
[ 300  690  851 1246]
[1217  707  722   85]
[870 722  85 999]
[1216    5  703  422]
[857]
[1021]
[789]
[670 672 186]
[1217  575  861]
[1385 1385  465]
[ 105 1023  670]
[ 672 1232  861]
[1217  876  937 1065]
[1217 1023  924]
[913]
[ 105 1023]
[568 680]
[1118   37  844]
[ 600 1246  415]
[1217  833 1281]
[1023]
[ 105 1023 1118  844]
[  37  600 1217  924]
[118 672   5]
[1381  651  300]
[1290]
[118 672   5 722]
[ 85 522 197]
[ 143 1217  218]
[ 105 1023  857  913]
[1380]
[1330  857  331 1246]
[ 833 1281  105 1023]
[1118 1118  672    5]
[ 651 1217  707  741]
[337 722  85 429]
[ 870 1122  871]
[789]
[722  85 207 429]
[1380]
[871]
[ 870 1003  871]
[ 37 415 817]
[ 694  996 1380]
[857]
[ 105 1023  857]
[1388 1045  672]
[  5  85 429]
[1380  707  741  337]
[789]
[ 857  844   37 1059]
[  37 1334  734  143]
[1217 1023  861 1125

[   5 1343  601 1332]
[1380 1380 1380]
[1217 1217  575 1030]
[ 651 1217  451 1336]
[1217  728  575 1030]
[1380]
[37  3]
[857  37]
[857 844]
[1388 1059 1122  844]
[ 672 1047 1064]
[ 924 1246  805]
[ 105 1023  857  844]
[  37 1059 1122  857]
[ 651 1217  218 1023]
[ 337 1221    5 1221]
[  5 722 672 735]
[722 117 455]
[1216    5  927]
[1023   37   37]
[1059 1287]
[ 857 1217  861 1246]
[1217 1217 1217  797]
[1217 1110  861]
[1380 1217 1110  861]
[ 143  143 1217  218]
[ 218  218 1023]
[1380]
[1118 1216    5]
[1217  861]
[1384]
[ 287 1216  861]
[ 150  568 1372]
[857]
[1380 1023]
[857]
[1118  844  672    5]
[ 115   37 1178 1246]
[ 287  833 1281]
[1380]
[1023]
[ 37 570]
[ 857 1118  672    5]
[ 85  85 927 924]
[ 857 1118 1388]
[1246 1217  707 1246]
[1217  707 1089 1124]
[ 651 1217  774 1388]
[1059   85   85 1003]
[ 924  118 1246]
[1217  218 1023]
[ 851 1216  924]
[1380]
[789]
[1329 1329 1343 1343]
[ 300  670 1217  924]
[1118  672    5  969]
[  37    1 1045]
[857  37  37]
[ 118 1118]
[  37 1059  

[836 672   5 841]
[672   5 841]
[  88 1217  999]
[672   5]
[1118  143 1217]
[1252 1023]
[1329]
[1118  672    5   88]
[1217 1129 1084 1225]
[ 857   88 1217 1130]
[1225  118  672    5]
[672   5 672   5]
[ 855 1329 1118  672]
[ 859 1217 1023]
[672   5]
[ 510 1336  600 1217]
[1136  118 1220 1336]
[ 600   85 1225  118]
[ 670 1216   85 1225]
[1216    5  690  851]
[735 722  85  72]
[1126  851  118 1221]
[   5   85 1269]
[ 651 1217  774]
[1380  465 1216]
[1216  861]
[118 672   5 927]
[857 334]
[1217 1217  928]
[1380]
[ 105 1023]
[857 844 672   5]
[1217 1225 1216   37]
[1334 1046  432]
[1118  672    5  722]
[ 85 904 672   5]
[ 651 1217  774  118]
[672   5 927]
[672   5 859]
[1217 1023  851]
[ 851 1217   72]
[1380]
[1023]
[150]
[ 857 1118  672    5]
[1217 1004  799  734]
[1225 1260 1217  218]
[ 722 1217  218  707]
[670 672 651 722]
[1374 1336 1045 1217]
[1225 1216    5  722]
[1246 1217 1023  851]
[1217  602 1227]
[1380]
[1217]
[1380]
[1023]
[465]
[857 844 672   5]
[ 857   88 1217   72]
[1217 121

[ 855 1232  670  855]
[1232  861  670  832]
[672   5 871 118]
[672 596 532 532]
[ 907  118 1220]
[162 105 537]
[1217 1040  580]
[857 715   5]
[244 672]
[735 722  85 254]
[254]
[568]
[788]
[1217  273]
[1232  861  670]
[112 832]
[672 670 732]
[1388  114  553 1217]
[832 862 571 857]
[672 670]
[1380]
[1232  861 1381  118]
[672 596  85 193]
[1261  118 1261]
[715   5 244]
[672 715   5]
[244 672  56]
[1217  892]
[1217  892]
[1217  892]
[1217  892]
[1217  892]
[1217  892]
[63]
[1232  670   85 1213]
[ 851  764 1011]
[1380]
[857]
[  84  683 1388]
[1388  561]
[1217  670  672  344]
[  85  344 1155 1300]
[622 282 236 207]
[ 193  870 1110]
[1155 1300  622  284]
[ 983 1110]
[ 193  870 1110]
[ 983 1110]
[1110]
[1380  983 1110]
[855 836]
[540]
[1216 1334 1217]
[ 193  861 1083]
[857]
[640]
[815 156]
[1216 1334 1217]
[1110  861]
[857]
[ 836 1217  195  861]
[855 857]
[1125]
[815]
[815 156]
[  37  397  817 1059]
[1217 1110  861]
[1388  561  102 1388]
[ 561 1388  331]
[836  37 242 817]
[ 672 1047 1330  526]

...

[672   5 735 722]
[ 85 722  85 964]
[ 479  537 1217]
[1023  590 1089]
[1385 1295 1217 1136]
[1380 1216  861]
[1217 1136 1380]
[ 857 1232  670  651]
[1217 1252  707  337]
[1217  871   74]
[ 610  859 1252]
[ 851 1217 1136]
[ 788 1385]
[  99 1217  479]
[ 672    5  651 1217]
[570 672]
[1380 1388 1044  672]
[1217   74  610]
[871 861]
[1232 1240  672    5]
[ 651 1217 1023 1089]
[ 851 1217 1051]
[1380   37 1044  672]
[834]
[855 672   5]
[764  99]
[870 841 672]
[  5 101 857]
[1232  528  672    5]
[ 85 728 199 900]
[ 651 1217  774  851]
[1217 1051 1221    5]
[ 722   85 1221    5]
[722  85 799]
[1380]
[672   5 672]
[  5 722]
[1217  747  799]
[1380]
[ 682 1023 1329]
[836]
[1023  859 1252  851]
[ 672 1221  670   85]
[1119 1126  851  497]
[ 651   85   37 1374]
[1045  722   85  722]
[ 85 183 300]
[672 596 722]
[ 532 1091]
[1385]
[1380 1216  861]
[857]
[ 857 1118 1232  670]
[1217  871  556]
[ 672    5 1246 1217]
[1023  851 1217  254]
[1252  851 1217 1051]
[1217 1030]
[1217 1030]
[1217  479]
[ 844  67

[1294  672]
[857]
[855]
[1023  422 1023  337]
[ 422  422 1217  207]
[1217  207  354]
[857]
[1388  600   85]
[207  72]
[  88  672 1221]
[  5  85 575]
[ 396  870 1122]
[1384]
[1217  575  861 1380]
[1380]
[1381  354  651 1217]
[1023  422  337]
[672   5  85]
[399 354 857]
[1380]
[  85 1126  851  354]
[857 672   5]
[1301 1089]
[788]
[1023]
[789]
[1217 1048  804  236]
[672   5 672]
[  5 672   5]
[1324 1217  196]
[789]
[789]
[857]
[1217  927  197]
[1346  131 1388]
[927 197]
[1290]
[672   5 859]
[1217 1023]
[ 37 530 672]
[682  85 791]
[ 672    5 1170]
[ 855  857 1125]
[207]
[789]
[118 672   5 422]
[789]
[672 735 722  85]
[799 870 354]
[ 672    5 1217  707]
[101 857]
[1384]
[ 37 570 672]
[ 672    5 1217]
[370 207  72]
[118 422]
[1294 1217  804]
[857 672   5 370]
[207  72 646 672]
[ 5 85 72]
[1380]
[ 844  857 1388  600]
[ 722 1285  197  722]
[ 85 207 197]
[1285  197]
[1294 1216 1388  600]
[ 207 1225]
[  85  207 1225 1380]
[1384   37 1065  672]
[ 101  857 1217]
[ 707 1089  927]
[1217  196  847]
[

[1089  851  672]
[101 614 651]
[718 575]
[1217  349  573 1365]
[1217  719  575  300]
[1217  326 1268]
[651 207]
[1106  999 1365 1287]
[1270  651  969]
[1217  414]
[1178 1246 1307]
[  31 1371]
[611 871]
[ 871  870 1003]
[1388    4  301]
[ 26   5 712]
[651 871]
[ 583 1365   85]
[840 718 575]
[1217  894   97]
[651 720 575]
[1217  402  651 1339]
[575 300 651]
[ 192 1217  718]
[118 370 575]
[894  97]
[ 99 718 575]
[718 575]
[402]
[1217  573  651]
[718 575]
[1221  131]
[1217  720  851  577]
[1217  720  851  577]
[ 326 1268  651  207]
[1217  611  651 1003]
[ 26   5 710]
[651 871]
[1324  999 1365]
[1217 1023  119]
[1269  859 1252]
[614 575 861]
[1269 1365 1023  119]
[1269  861 1252]
[969]
[ 999 1380  969 1365]
[614 718 575]
[1217   72  651 1324]
[871 614]
[1217  349  573  651]
[1217  719  575]
[1217  720  851  575]
[1385]
[326 207]
[1217  614  651 1003]
[ 349  573  651 1217]
[720 851 577]
[1217  402  651  575]
[611 651 871]
[ 614  651 1003]
[ 349  573  651 1217]
[720 851 577]
[1217 1287 1270]


[ 532 1270  750]
[857 857 719]
[1214 1217  575]
[1380]
[672   5]
[1023]
[ 672    5  719 1214]
[1217  898  575]
[672   5  85]
[306 851 898]
[ 575  118 1381]
[857]
[670  85]
[857]
[672   5 719]
[1214  898  575]
[306 851 898]
[ 575  118 1381]
[1380]
[ 37   6 288 672]
[672   5 575]
[ 457 1073]
[857]
[898 575]
[672   5 969]
[651 300]
[ 672  735  722 1315]
[1393  859 1217 1252]
[118 735 722  85]
[597 851  85 692]
[ 672    5 1110]
[ 651 1101]
[836 672   5]
[ 195  651 1101]
[672   5  85 788]
[722  85 969 651]
[300 118 672  85]
[1317  548 1013 1073]
[857]
[672   5 207 651]
[300 718 207]
[672   5 746 851]
[1269  118   85 1145]
[1252 1269  118  859]
[1217 1023  741  337]
[ 670   85 1145]
[1232  861]
[1380]
[ 672    5   85 1104]
[207 651 300 841]
[ 465 1104  207]
[672   5  85 560]
[1073  537  741  451]
[ 118  859 1217 1252]
[ 670   85 1269 1380]
[857]
[1232  861]
[672   5 969 651]
[300 672   5]
[927 651 300]
[1110  651 1101]
[118 735 722  85]
[ 597  118 1220 1388]
[600  85 746 851]
[ 851 1118  753

[1136   46]
[ 859 1246 1389 1023]
[1269  118   85]
[ 667 1269]
[672   5  85  85]
[ 69 207 947]
[597 641 479]
[ 455  859 1217 1252]
[927]
[672   5  85]
[947 840]
[ 927  651 1217  259]
[ 84 683]
[ 859 1246 1389  218]
[1236  944 1270]
[575]
[1110  189   72]
[672   5  85  72]
[863  72]
[1324]
[ 859 1389 1252]
[  72 1365   85]
[ 944 1269]
[ 859 1217 1023  218]
[1302  422 1270]
[672   5 575 300]
[ 859 1246 1389]
[1023 1252]
[947 840 207]
[672   5 859]
[1389 1023  533]
[792  42 927]
[ 859 1246 1389]
[707 218]
[ 532  944 1270]
[1381]
[ 422 1221]
[ 532  944 1269]
[1380 1381  859 1246]
[1389  218  707]
[1343    5 1217  300]
[1381  300]
[1381  300  857 1125]
[1384]
[ 859 1246 1389  707]
[ 707  218 1217]
[ 207 1073 1347]
[ 722   85 1130]
[189  46]
[190  46]
[ 859 1246 1389 1252]
[  85 1269  118   85]
[1145 1003  300]
[1252 1023]
[672   5  85]
[  72 1324  300]
[  72 1365   85  944]
[1269  143 1217  259]
[1324]
[ 85 207 300]
[1380  207]
[  37 1152]
[855]
[855]
[  37    3 1125]
[ 115   37 1178]
[1246